# Primitive Types

`Python has a number of built-in types: numerics (e.g., integer), sequences (e.g., list), mappings (e.g., dict), as well as classes, instances and exceptions. All instances of these types are objects. Integers in Python3 are unbounded-the maximum integer representable is a fr:nction of the available memory. The constant sys.maxsize can be used to find the word-size; specifically, it's the maximum value integer that can be stored in the word, e.g., 2**63 - 1 on a 64-bit machine.` 

* `Be very familiar with the bit-wise operators such as 6&4,11 Z, B>>L, -L6>>2, 1<<10, -0, 15^x. Negative numbers are treated as their 2's complement value. (There is no concept of an unsigned shift in Python, since integers have infinite precision.)`
* `The key methods for numeric types are abs(-34.5), math. ceil(2. 17), math. floor(3. 14), min(x, -4), max(3 .14, y), pow(2 .7L, 3.14) (altemately, 2.71 ** 3.14), and math. sqrt (22 5).`
* `Know how to interconvert integers and strings, e.g., str(421 , int('42'), floats and strings, e.g., str(3. 14), float(' 3. 14').`
* `Unlike integers, floats are not infinite precision, and it's convenient to refer to infinity as float('inf ') and float('-inf '). These values are comparable to integers, and can be used to create psuedo max-int and pseudo min-int.`
* `When comparing floating point values consider using math.iscloseO-it is robust, e.g., when comparing very large values, and can handle both absolute and relative differences.`
* `The key methods in random are random.randrange(28), random.randint(8,1.6), randon. randomO, random. shuffle (A), and random. choice(A).`



### Q1. Computing the parity of the word

`The parity of a binary word is 1 if the number of 1s in the word is odd; otherwise, it is 0. For example, the parity of 1011 is 1, and the parity of 10001000 is 0. Parity checks are used to detect single bit errors in data storage and communication. It is fairly straightforward to write code that computes the parity of a single 64-bit word.`


#### Solution1
`The brute-force algorithm iteratively tests the value of each bit while tracking the number of 1s seen so far. Since we only care if the number of 1s is even or odd, we can store the number mod 2.`

In [1]:
def parity(x):
    result=0
    while x:
        result^=x&1 #x&1 computes the last bit of x we will xor it with result to check parity 
        x>>=1 #changing x 
    return result    

In [2]:
%%timeit
li=[parity(i) for i in range(100000)]

386 ms ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__`The time complexity is O(n), where n is the word size.`__

#### Solution2
`The first improvement is based on erasing the lowest set bit in a word in a single operation, thereby improving performance in the best- and average-cases`

` x &(x * 1) equals x with its lowest set bit erased. (Here &isthebitwise-ANDoperator.) 1 Forexample, if x= (00101100)2,thenx-1= (00101011)2, so x &(x - 1) = (00101100)2 & (00101011)2 = (00101000)2. This fact can be used to reduce the time complexity.`

In [3]:
def parity1(x):
    result=0
    while x:
        result^=1
        x&=x-1 # drops the lowest set bit of x or another trick is x & `(x-1) it isolates the lowest bit that is 1 in x
    return result    

 __`Let k be the number of bits set to 1 in a particular word. (For example, for 10001010, k = 3.) Then time complexity of the algorithm below is O(k)`__ 

In [4]:

%%timeit
li1=[parity1(i) for i in range(100000)]

178 ms ± 8.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Solution3
`We demonstrate caching. Clearly, we cannot cache the parity of every 64-bit integer-we would need 2e bits of storage, which is of the order of two exabytes. However, when computing the parity of a collection of bits, it does not matter how we group those bits, i.e., the computation is associative. Therefore, we can compute the parity of a 64-bit integer by grouping its bits into four nonoverlapping 16 bit subwords, computing the parity of each subwords, and then computing the parity of these four subresults. We choose 16 since 216 = 65536 is relatively small, which makes it feasible to cache the parity of all 16-bit words using an array. Furthernore, since 16 evenly divides 64`

`To lookup the parity of the first two bits in (11101010), we right shift by 6, to get (00000011), and use this as an index into the cache. To lookup the parity of the next two bits, i.e., (10), we right shift by 4, to get (10) in the two least-significant bit places. The right shift does not remove the leading (11)-it results in (00001110). We cannot index the cache with this, it leads to an out-of-bounds access. To get the last two bits after the right shift by 4, we bitwise-AND (00001110) with (0000001 1) (this is the "mask" used to extract the last 2 bits). The result is (00000010). Similar masking is needed for the two other 2-bit lookups.`

In [5]:
def parity2(x):
    MASK_SIZE=16
    BIT_MASK=0xFFFF
    return(Precomputed_parity[x>>(MASK_SIZE*3)]^Precomputed_parity[(x>>(MASK_SIZE*2))&BIT_MASK]^Precomputed_parity[(x>>(MASK_SIZE))&BIT_MASK]^Precomputed_parity[x&BIT_MASK])


`The time complexity is a function of the size of the keys used to index the lookup table. Let L be the width of the words for which we cache the results, and n the word size. Since there are n/L terms, the time complexity is O(n/L), assuming word-level operations, such as shifting, take O(1) time. (This does not include the time for initialization of the lookup table.) `

#### Solution4
`We can improve on the O(n) worst-case time complexity of the previous algorithms by exploiting some simple properties of parity. Specifically, the XOR of two bits is defined to be 0 if both bits are 0 or both bits are 1; otherwise it is 1. XOR has the property of being associative, i.e., it does not matter how we group bits, as well as commutative, i.e., the order in which we perform the XORs does not change the result. The XOR of a group of bits is its parity. We can exploit this fact to use the CPU's word-level XOR instruction to process multiple bits at a tirne. `

`For example, the parity of (b63,b62,. .. ,b3,b2, b1, b0) equals the parity of the XOR of (b63b62,. . . ,b32) and (b31, bn,. .., b0). The XOR of these two 32-bit values can be computed with a single shift and a shgle 32-bit XOR instruction. We repeat the same operation on32-,16-,8-, 4-, 2-, and 1-bit operands to get the final result. Note that the leading bits are not meaningful, and we have to explicitly extract the result from the least-significant bit. We illustrate the approach with an 8-bit word. The parity of (11010111) is the same as the parity of (1101) XORed with (0111), i.e., of (1010). This in tum is the same as the parity of (10) XORed with (10), i.e., of (00). The final result is the XOR of (0) with (0), i.e.,0, Note that the first XOR yields (11011010), and only the last 4 bits are relevant going forward. The second XOR yields (11101100), and only the last 2 bits are relevant. The third XOR yields (10011010). The last bit is the result, and to extract it we have to bitwise-AND with (00000001).`


In [6]:
def parityOp(x):
    x ^= x >> 32
    x ^= x >> 16
    x ^= x >> 8
    x ^= x >> 4
    x ^= x >> 2
    x ^= x >> 1
    return x & 0x1


In [7]:

%%timeit
li2=[parityOp(i) for i in range(100000)] #best algorithm in terms of space or time constraints

89 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


`The time complexity is `__`O(logn)`__`, where n is the word size. Note that we can combine caching with word-level operations, e.g. by doing a lookup in the XOR-based approach once we get to 16 bits. The actual runtimes depend on the input data, e.g., the refinement of the brute-force algorithm is very fast on sparse inputs. However, for random inputs, the refinement of the brute-force is roughly 20% faster than the brute-force algorithm. The table-based approach is four times faster still, and using associativity reduces run time by another factor of two.`

#### Similar Questions

___Right Propagate the rightmost set bit in x eg. (01010000)2 to (01011111)2___

In [8]:
def rightpropagate(x):
    a=x&(x-1)
    b=x&(~(x-1))
    c=b-1
    d=a|b|c
    return bin(d)

In [9]:
rightpropagate(0b101000100)

'0b101000111'

___Compute x mod a power of 2 eg. 77 mod 64=13___

In [10]:
def modpow2(x):
    y=x
    while(x&(x-1)):
        x&=(x-1)
    return y-x

In [11]:
modpow2(43)

11

___Test if x is a power of 2,i.e.for 1 2 4..,evaluates to true___

In [12]:
def ispowof2(n): 

 
    n1=n
    n |= n>>1

    n |= n>>2

    n |= n>>4
    n |= n>>8
    n |= n>>16
    n |= n>>32 #for 64 bit word

    n = n + 1

 
    return "Yes" if (n1-(n >> 1)==0) else "No"


n = 2**63
print(ispowof2(n)) 

 

Yes


### Q2. Swap Bits